In [1]:
!conda install -y gdown

Retrieving notices: ...working... done
done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 24.4.0

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.4.0



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - gdown


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    filelock-3.14.0            |     pyhd8ed1ab_0          16 KB  conda-forge
    gdown-5.1.0                |     pyhd8ed1ab_0          21 KB  conda-forge
    openssl-3.3.0              |       hd590300_0         2.8 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.8 MB

The following NEW packages will be 

In [2]:
!gdown --id 1LB53KldZ_hG_Kc2wEBmRjLH0Js80WGgr

/opt/conda/lib/python3.10/site-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1LB53KldZ_hG_Kc2wEBmRjLH0Js80WGgr
From (redirected): https://drive.google.com/uc?id=1LB53KldZ_hG_Kc2wEBmRjLH0Js80WGgr&confirm=t&uuid=483f3e26-c4fd-431f-a509-dbdd1b8a4162
To: /kaggle/working/trainval.zip
100%|█████████████████████████████████████████| 756M/756M [00:07<00:00, 102MB/s]


In [3]:
!unzip trainval.zip

Archive:  trainval.zip
   creating: trainval/
   creating: trainval/A/
  inflating: trainval/A/0000.png     
  inflating: trainval/A/0001.png     
  inflating: trainval/A/0002.png     
  inflating: trainval/A/0003.png     
  inflating: trainval/A/0004.png     
  inflating: trainval/A/0005.png     
  inflating: trainval/A/0006.png     
  inflating: trainval/A/0007.png     
  inflating: trainval/A/0008.png     
  inflating: trainval/A/0009.png     
  inflating: trainval/A/0010.png     
  inflating: trainval/A/0011.png     
  inflating: trainval/A/0012.png     
  inflating: trainval/A/0013.png     
  inflating: trainval/A/0014.png     
  inflating: trainval/A/0015.png     
  inflating: trainval/A/0016.png     
  inflating: trainval/A/0017.png     
  inflating: trainval/A/0018.png     
  inflating: trainval/A/0019.png     
  inflating: trainval/A/0020.png     
  inflating: trainval/A/0021.png     
  inflating: trainval/A/0022.png     
  inflating: trainval/A/0023.png     
  inflating: trai

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn.functional as F
from skimage import io, color

In [5]:
# Load images from folders A and B
folder_A = 'trainval/A'
folder_B = 'trainval/B'
folder_label = 'trainval/label'

In [6]:
device = ("cuda" if torch.cuda.is_available() else "cpu") # Use GPU or CPU for training

In [7]:
print(device)

cuda


In [8]:
# Function to load images from directory
def load_images_from_folder(folder, is_gray = True):
    images = []
    for filename in os.listdir(folder):
        img = io.imread(os.path.join(folder,filename))
        if img is not None:
            if is_gray:
                images.append(color.rgb2gray(img / 255))
            else:    
                images.append(img / 255)
    return images


In [9]:
images_A = load_images_from_folder(folder_A, is_gray=False)


In [10]:
images_B = load_images_from_folder(folder_B, is_gray=False)


In [11]:
labels = load_images_from_folder(folder_label, is_gray = False)

In [12]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5, 0.5),
])

In [13]:
class ChangeDetectionDataset(Dataset):
    def __init__(self, images_A, images_B, labels, transform=None):
        self.images_A = images_A
        self.images_B = images_B
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image_A = self.images_A[idx]
        image_B = self.images_B[idx]
        label = self.labels[idx]

        if self.transform:
            image_A = self.transform(image_A)
            image_B = self.transform(image_B)
            label = self.transform(label)

        return image_A, image_B, label

In [14]:
dataset = ChangeDetectionDataset(images_A, images_B, labels, transform=transform)

In [15]:
del images_A, images_B, labels

In [16]:
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

In [17]:
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

In [18]:
del dataset

In [19]:
batch_size = 16

In [20]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [21]:
del train_dataset,val_dataset

In [64]:
import torch
import torch.nn as nn
import torch.nn.functional as F  # Added import for functional module
from torch.nn.functional import relu, sigmoid

class ChangeDetectionUNet(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Encoder
        self.e11 = nn.Conv2d(6, 64, kernel_size=3 , padding=1)
        self.e12 = nn.Conv2d(64, 64, kernel_size=3 , padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.e21 = nn.Conv2d(64, 128, kernel_size=3 , padding=1)
        self.e22 = nn.Conv2d(128, 128, kernel_size=3 , padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.e31 = nn.Conv2d(128, 256, kernel_size=3 , padding=1)
        self.e32 = nn.Conv2d(256, 256, kernel_size=3 , padding=1)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.e41 = nn.Conv2d(256, 512, kernel_size=3 , padding=1)
        self.e42 = nn.Conv2d(512, 512, kernel_size=3 , padding=1)
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.e51 = nn.Conv2d(512, 1024, kernel_size=3 , padding=1)
        self.e52 = nn.Conv2d(1024, 1024, kernel_size=3 , padding=1)

        # Decoder
        self.upconv1 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.d11 = nn.Conv2d(1024, 512, kernel_size=3 , padding=1)
        self.d12 = nn.Conv2d(512, 512, kernel_size=3 , padding=1)

        self.upconv2 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.d21 = nn.Conv2d(512, 256, kernel_size=3 , padding=1)
        self.d22 = nn.Conv2d(256, 256, kernel_size=3 , padding=1)

        self.upconv3 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.d31 = nn.Conv2d(256, 128, kernel_size=3 , padding=1)
        self.d32 = nn.Conv2d(128, 128, kernel_size=3 , padding=1)

        self.upconv4 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.d41 = nn.Conv2d(128, 64, kernel_size=3 , padding=1)
        self.d42 = nn.Conv2d(64, 64, kernel_size=3 , padding=1)

        # Output layer
        self.outconv = nn.Conv2d(64, 1, kernel_size=1)

    def forward(self, x1, x2):
        
        # Concatenate input images along the channel dimension
        x = torch.cat([x1, x2], dim=1)

        # Encoder
        xe11 = torch.relu(self.e11(x))
        xe12 = torch.relu(self.e12(xe11))
        xp1 = self.pool1(xe12)


        xe21 = torch.relu(self.e21(xp1))
        xe22 = torch.relu(self.e22(xe21))
        xp2 = self.pool2(xe22)


        xe31 = torch.relu(self.e31(xp2))
        xe32 = torch.relu(self.e32(xe31))
        xp3 = self.pool3(xe32)


        xe41 = torch.relu(self.e41(xp3))
        xe42 = torch.relu(self.e42(xe41))
        xp4 = self.pool4(xe42)


        xe51 = torch.relu(self.e51(xp4))
        xe52 = torch.relu(self.e52(xe51))


        # Decoder
        xu1 = self.upconv1(xe52)
        # xe42_resized = F.interpolate(xe42, size=xu1.size()[2:], mode='bilinear', align_corners=True)
        xu11 = torch.cat([xe42, xu1], dim=1)
        xd11 = torch.relu(self.d11(xu11))
        xd12 = torch.relu(self.d12(xd11))

        xu2 = self.upconv2(xd12)
        # xe32_resized = F.interpolate(xe32, size=xu2.size()[2:], mode='bilinear', align_corners=True)
        xu22 = torch.cat([xe32, xu2], dim=1)
        xd21 = torch.relu(self.d21(xu22))
        xd22 = torch.relu(self.d22(xd21))

        xu3 = self.upconv3(xd22)
        # xe22_resized = F.interpolate(xe22, size=xu3.size()[2:], mode='bilinear', align_corners=True)
        xu33 = torch.cat([xe22, xu3], dim=1)
        xd31 = torch.relu(self.d31(xu33))
        xd32 = torch.relu(self.d32(xd31))

        xu4 = self.upconv4(xd32)
        # xe12_resized = F.interpolate(xe12, size=xu4.size()[2:], mode='bilinear', align_corners=True)
        xu44 = torch.cat([xe12, xu4], dim=1)
        xd41 = torch.relu(self.d41(xu44))
        xd42 = torch.relu(self.d42(xd41))

        # Output layer with sigmoid activation
#         out = torch.sigmoid(self.outconv(xd42))

        
        return self.outconv(xd42)


In [65]:
model = ChangeDetectionUNet()
model = model.to(device)

In [66]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.0005)

In [69]:
# Training loop
num_epochs = 3



best_losses = float('inf')

for epoch in range(num_epochs):
    
    total_true = []
    total_pred = []
    
    model.train()
    total_train_loss = 0
    total_intersection = 0
    total_union = 0
    
    for batch_idx, (data_A, data_B, target) in enumerate(train_loader):
        print("Batch index train:",batch_idx)
        optimizer.zero_grad()
        data_A = data_A.to(device=device, dtype=torch.float32, memory_format=torch.channels_last)
        data_B = data_B.to(device=device, dtype=torch.float32, memory_format=torch.channels_last)
        target = target.to(device=device, dtype=torch.long)
        output = model(data_A, data_B)
        loss = criterion(output, target.float())
        loss.backward()
        optimizer.step() 
        total_train_loss += loss.item()



    # Validation
    model.eval()
    total_val_loss = 0
    total_val_intersection = 0
    total_val_union = 0
    
    with torch.no_grad():
        for batch_idx, (data_A, data_B, target) in enumerate(val_loader):
            print("Batch index test:",batch_idx)
            data_A = data_A.to(device=device, dtype=torch.float32, memory_format=torch.channels_last)
            data_B = data_B.to(device=device, dtype=torch.float32, memory_format=torch.channels_last)
            target = target.to(device=device, dtype=torch.long)
            output = model(data_A, data_B)
            val_loss = criterion(output, target.float())
            total_val_loss += val_loss.item()
            
#             out_binary = torch.where(output > 0.5, torch.tensor(1.0), torch.tensor(0.0))

            total_pred.append(output)
            total_true.append(target)

    if(total_val_loss < best_losses):
        best_losses = total_val_loss
        torch.save(model.state_dict(), 'model_10.pickle')
    
   
        
    print(f'Epoch {epoch+1}, Train Loss: {total_train_loss/len(train_loader)}'
          f'Validation Loss: {total_val_loss/len(val_loader)}')


Batch index train: 0


RuntimeError: result type Float can't be cast to the desired output type Long

In [ ]:
def calculate_jaccard_index(pred_mask, true_mask):
    intersection = np.logical_and(pred_mask, true_mask).sum()
    union = np.logical_or(pred_mask, true_mask).sum()
    jaccard_index = intersection / union
    return jaccard_index

In [61]:
temp = total_pred

In [54]:

# for i in range(len(temp)):
#     temp[i] = temp[i].to('cpu')
#     temp[i] = temp[i] > 0

In [62]:
temp[0][0]

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]], device='cuda:0')

In [36]:
import matplotlib.pyplot as plt
import numpy as np

In [63]:
for i in range(len(temp)):
    for j in range(len(temp[i])):
        for pixel in temp[i][j]:
            for p in pixel:
                for c in p:
                    if c == 1:
                        print("Found")

KeyboardInterrupt: 